<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Imports<strong></font></h1>

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report
)

from sklearn.ensemble import (
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, RandomForestClassifier
)

from sklearn.linear_model import (
    LogisticRegression, PassiveAggressiveClassifier, Perceptron,
    RidgeClassifier, RidgeClassifierCV, SGDClassifier
)

from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
)

from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.dummy import DummyClassifier
from sklearn.calibration import CalibratedClassifierCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler

warnings.filterwarnings(action='ignore')
%matplotlib inline
plt.style.use('ggplot')


<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Data Overview<strong></font></h1>

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')

In [3]:
test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [4]:
train.shape,test.shape

((3116945, 22), (2077964, 21))

In [5]:
train.head(5)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [6]:
train.isna().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Dropped Columns with high Na values<strong></font></h1>

In [7]:
# droping those columns which are having more than 100k missing values
train.drop(columns = ['id','cap-surface','gill-attachment','gill-spacing','stem-root','stem-surface','veil-type','veil-color','spore-print-color','ring-type'],inplace = True)

In [8]:
train.isna().sum()

class                    0
cap-diameter             4
cap-shape               40
cap-color               12
does-bruise-or-bleed     8
gill-color              57
stem-height              0
stem-width               0
stem-color              38
has-ring                24
habitat                 45
season                   0
dtype: int64

<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Pipeline<strong></font></h1>

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [10]:
numerical_column = ['cap-diameter','stem-height','stem-width']
categorical_column = ['cap-shape','cap-color','does-bruise-or-bleed','gill-color','stem-color','has-ring','habitat','season']

In [11]:
Numeric_Pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

Categorical_Pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [12]:
Final_Pipeline = ColumnTransformer(transformers=[('Numeric_Pipeline',Numeric_Pipeline,numerical_column),
                                                ('Categorical_Pipeline',Categorical_Pipeline,categorical_column)
                                                ])

In [13]:
from sklearn import set_config
set_config(display='diagram')


<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Data Splitting<strong></font></h1>

In [14]:
from sklearn.model_selection import train_test_split
X = train.drop(['class'],axis = 1)
y = train['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 7)

<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Final Transformed Data<strong></font></h1>

In [15]:
X_train_transformed = Final_Pipeline.fit_transform(X_train)
X_test_transformed = Final_Pipeline.transform(X_test)

In [16]:
y_train = np.where(pd.get_dummies(y_train, drop_first=True) == True , 1, 0)
y_test = np.where(pd.get_dummies(y_test, drop_first = True) == True , 1, 0)

In [17]:
X_train_transformed.shape, X_test_transformed.shape, y_train.shape, y_test.shape

((2493556, 343), (623389, 343), (2493556, 1), (623389, 1))

<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Models<strong></font></h1>
    <font color=#000000><strong>Since training and testing data is very large i have only used XGBClassifier and Decisiojn Tree Classifier , one can check other models also (just comment out ) . <strong></font>

In [18]:
models = {
    
  
#     "LabelSpreading": LabelSpreading(),
#     "LGBMClassifier": LGBMClassifier(),
#     "LinearSVC": LinearSVC(),
#     "CalibratedClassifierCV": CalibratedClassifierCV(),
#     "RandomForestClassifier": RandomForestClassifier(),
#     "SGDClassifier": SGDClassifier(),
#     "LogisticRegression": LogisticRegression(),
#     "KNeighborsClassifier": KNeighborsClassifier(),
#     "Perceptron": Perceptron(),
#     "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
#     "SVC": SVC(),
#     "GaussianNB": GaussianNB(),
#     "BernoulliNB": BernoulliNB(),
    "XGBClassifier": XGBClassifier(),
#     "AdaBoostClassifier": AdaBoostClassifier(),
#     "BaggingClassifier": BaggingClassifier(),
#     "NuSVC": NuSVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),

}

results = {
    "Model": [],
    "Accuracy": []
    
}


for model_name, model in models.items():

    model.fit(X_train_transformed, y_train)
    
    y_pred = model.predict(X_test_transformed)
    
    results["Model"].append(model_name)
    results["Accuracy"].append(accuracy_score(y_test, y_pred))

results_df = pd.DataFrame(results)

In [19]:
results_df

,Model,Accuracy
0,XGBClassifier,0.975622
1,DecisionTreeClassifier,0.966937


<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Conclusion<strong></font></h1>
    <font color=#000000><strong>1. I have only checked 2 models , one can check others models <br> 2. This notebook is not complete i will add rest of my work in next version <strong></font>

<div style = "background-color: #dfe6e3; font-size:150%; text-align:left; border: 7px solid #3449eb; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);font-family : calibri ; border-radius : 20px 20px 20px 20px; padding: 10px; width:95%">
    
<h1 align="center"><font color=#FF0000><strong>Request<strong></font></h1>
    <font color=#000000><strong> I tried checking the models on subset of data but even after using only 100k instances model training is very slow , please comment your suggestions , i will definetly try those  <strong></font>


<h3 style="text-align:center;">Please Upvote this notebook if you found it helpful</h3>
<img src="https://media1.giphy.com/media/2lQCBjuFMLCOvXno4l/giphy.gif" alt="GIF" style="height: 300px; width: auto;margin:auto;">
